In [5]:
#This script takes a initial date and final date and downloads the files needed to compute the wave activity flux (WAF)
#for that given period. NCEP/NCAR Reanalysis (Kalnay etal 1996)are used 

#libraries needed
import urllib.request
from bs4 import BeautifulSoup
import numpy as np

def clean():   #clean enviroment
    import os
    os.system("rm -f /tmp/*.gz /tmp/*.nc")
    
def descarga_nc( mesi, diai, mesf, diaf, aniof, variable_entrada, variable_salida, tipo):
    #tipo controla que dato bajo: 2: para anomalia 3 para climatologia
    # Open NCEP NCAR to access to lik to data

        url = 'http://www.esrl.noaa.gov/psd/cgi-bin/data/composites/comp.day.pl?var='+variable_entrada+'&level=250mb&iy[1]=&im[1]=&id[1]=&iy[2]=&im[2]=&id[2]=&iy[3]=&im[3]=&id[3]=&iy[4]=&im[4]=&id[4]=&iy[5]=&im[5]=&id[5]=&iy[6]=&im[6]=&id[6]=&iy[7]=&im[7]=&id[7]=&iy[8]=&im[8]=&id[8]=&iy[9]=&im[9]=&id[9]=&iy[10]=&im[10]=&id[10]=&iy[11]=&im[11]=&id[11]=&iy[12]=&im[12]=&id[12]=&iy[13]=&im[13]=&id[13]=&iy[14]=&im[14]=&id[14]=&iy[15]=&im[15]=&id[15]=&iy[16]=&im[16]=&id[16]=&iy[17]=&im[17]=&id[17]=&iy[18]=&im[18]=&id[18]=&iy[19]=&im[19]=&id[19]=&iy[20]=&im[20]=&id[20]=&monr1='+str(mesi)+'&dayr1='+str(diai)+'&monr2='+str(mesf)+'&dayr2='+str(diaf)+'&iyr[1]='+str(aniof)+'&filenamein=&plotlabel=&lag=0&labelc=Color&labels=Shaded&type='+str(tipo)+'&scale=&label=0&cint=&lowr=&highr=&istate=0&proj=ALL&xlat1=&xlat2=&xlon1=&xlon2=&custproj=Cylindrical+Equidistant&level1=1000mb&level2=10mb&Submit=Create+Plot'
        response = urllib.request.urlopen(url)
        data = response.read()      # a `bytes` object

        soup = BeautifulSoup(data,'html.parser') #como es un xml, beautifull tiene un modulo de eso
        link = soup.findAll('img')[-1]['src']
        #Esta es una manera en que podemos obtener el link del .nc que tiene los datos del grafico

        link=list(link)
        link[-3]='n'
        link[-2]='c'
        link[-1]=''
        
        #get nc file save as netcdf
        ruta = "./tmp/"
        urllib.request.urlretrieve('http://www.esrl.noaa.gov'+"".join(link), ruta+variable_salida+'.nc')
    
        

clean()
#initial date

iniy = 2016
inim = 8
inid = 1

#final date
finy = 2016
finm = 8
find = 10
#download: geopotential height, u and v climatology for the selected period
#          geopotential height anomalies

variables = ['Zonal+Wind','Meridional+Wind','Geopotential+Height']

out_var = ['zonalw_climo','meridw_climo','hgt_climo','hgt']

for i in range(0,3):
    if i==2: # download climo and anomaly for geopotential
        descarga_nc(inim,inid,finm,find,finy,variables[i],out_var[i],3)
        descarga_nc(inim,inid,finm,find,finy,variables[i],out_var[i+1],2)
    descarga_nc(inim,inid,finm,find,finy,variables[i],out_var[i],3)




In [20]:
# manipulation of netCDF files
from netCDF4 import Dataset

def manipular_nc(archivo,variable):
    dataset = Dataset(archivo, 'r')
    var_out = dataset.variables[variable][:]
    lon = dataset.variables['lon'][:]
    lat = dataset.variables['lat'][:]
    dataset.close()
    return var_out, lon, lat

ruta = "./tmp/"

out_var = ['zonalw_climo','meridw_climo','hgt_climo','hgt']

nc_var = ['uwnd','vwnd','hgt']

[uclm,lon,lat] = manipular_nc(ruta+out_var[0]+'.nc',nc_var[0])

[vclm,lon,lat] = manipular_nc(ruta+out_var[1]+'.nc',nc_var[1])

[zclm,lon,lat] = manipular_nc(ruta+out_var[2]+'.nc',nc_var[2])

[zaa,lon,lat] = manipular_nc(ruta+out_var[3]+'.nc',nc_var[2])  #zaa [1 nlat nlon]

#restringe domain to latitudes south to 0
index_lat = np.where(lat<0)
zaa = zaa[0,index_lat,:]
lat = lat[index_lat]
uclm = uclm[0,index_lat,:]
vclm = vclm[0,index_lat,:]

#print(range(index_lat))
print(lat.shape)
print(zaa.shape)
print(uclm.shape)
print(vclm.shape)


(36,)
(1, 36, 144)
(1, 36, 144)
(1, 36, 144)


In [36]:
#computation of plum fluxes. Script adapted from Kazuaki Nishii and Hisashi Nakamura
import math
import numpy as np

def c_diff(arr, h, dim, cyclic = False):  #compute derivate of array variable respect to h associated to dim
    #adapted from kuchaale scrip
    ndim = arr.ndim
    lst = [i for i in range(ndim)]

    lst[dim], lst[0] = lst[0], lst[dim]
    rank = lst 
    arr = np.transpose(arr, tuple(rank))

    if ndim == 3:
        shp = (arr.shape[0]-2,1,1)
    elif ndim == 4:
        shp = (arr.shape[0]-2,1,1,1)
    
    d_arr = np.copy(arr)
    if not cyclic:  
        d_arr[0,...] = (arr[1,...]-arr[0,...])/(h[1]-h[0])
        d_arr[-1,...] = (arr[-1,...]-arr[-2,...])/(h[-1]-h[-2])
        d_arr[1:-1,...] = (arr[2:,...]-arr[0:-2,...])/np.reshape(h[2:]-h[0:-2], shp)

    elif cyclic:
        d_arr[0,...] = (arr[1,...]-arr[-1,...])/(h[1]-h[-1])
        d_arr[-1,...] = (arr[0,...]-arr[-2,...])/(h[0]-h[-2])
        d_arr[1:-1,...] = (arr[2:,...]-arr[0:-2,...])/np.reshape(h[2:]-h[0:-2], shp)

    d_arr = np.transpose(d_arr, tuple(rank))

    return d_arr

[xxx,nlats,nlons] = zaa.shape

#gas constant
Ra = 290

#earth radius
a = 6400000

coslat = np.cos(lat*3.14/180)
sinlat = np.sin(lat*3.14/180)

#Coriolis parameter
f = 2*7.24/100000*sinlat
f0 = 2*7.24/100000*math.sin(43*3.14/180)

#gravity
g = 9.8

# unit [Pa]
lev = 20000

# basic state (climatology): uclm vclm zclm

# anomalies zaa

# QG stream function
psiaa = g/np.transpose(np.tile(f,(nlons,1)))*zaa

# magnitude of basic state wind speed
magU = np.sqrt(np.add(np.power(uclm,2),np.power(vclm,2)))

#psi derivatives
dpsidlon = c_diff(psiaa,lon,2)
ddpsidlonlon = c_diff(dpsidlon,lon,2)

dpsidlat = c_diff(psiaa,lat,1)
ddpsidlatlat = c_diff(dpsidlat,lat,1)
ddpsidlatlon = c_diff(dpsidlat,lon,2)

termxu = dpsidlon*dpsidlon-psiaa*ddpsidlonlon

termxv = dpsidlon*dpsidlat-ddpsidlatlon*psiaa

termyv = dpsidlat*dpsidlat-psiaa*ddpsidlatlat

# "p" is normalized by 1000hPa
coeff1=np.transpose(np.tile(coslat,(nlons,1)))*(lev/100000)/(2*magU)
#x-component

px = coeff1/(a*a*np.transpose(np.tile(coslat,(nlons,1))))*( uclm*termxu/np.transpose(np.tile(coslat,(nlons,1))) + vclm*termxv)

#y-component

py = coeff1/(a*a)*( uclm/np.transpose(np.tile(coslat,(nlons,1)))*termxv + vclm*termyv)

#set lon 0 360

#set gxout contour
#set cint 30
#set black -0.1 0.1

#* stream-function-like geopotential height
#d maskout( zaa*abs(f0/f),  abs(lat)-10)

#* horizontal wave-activity flux
#set arrscl 0.5 20
#d skip(px,6,4);maskout( py , abs(lat)-10)

(1, 5184)
(1, 36, 144)


(36,)
(1, 36, 144)
